In [56]:
import librosa
import os, gc
import numpy as np
import random

from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score

import xgboost as xgb

In [103]:
def extractMfccFeature(filepath):
    try:
        signal, fs = librosa.load(file_path, offset=12, duration=6)
        mfccs = librosa.feature.mfcc(signal, sr=fs, n_mfcc=12)
        l = mfccs.shape[1]
        mfccs = mfccs[:, int(0.2 * l):int(0.8 * l)]

        mean = mfccs.mean(axis=1)
        covar = np.cov(mfccs, rowvar=1)

        mean.resize(1, mean.shape[0])
        # it returns matrix.. not useful for machine learning algorithms except KNN
        npArray = np.concatenate((mean, covar), axis=0)
        npArray = npArray.flatten()
        return npArray
    except Exception as e:
        print ("Error accured" + str(e))
        return None

In [48]:
%%time
walk_dir = "genres"

Y = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
     'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

all_input_data = []
all_input_class = []
for root, subdirs, files in os.walk(walk_dir):
    print("Walk on dir:",root,'Total File:',len(files))
    for filename in files:
        if filename.endswith('.au'):
            file_path = os.path.join(root, filename)
            ppFileName = file_path.replace(".au",".pp")
            feature = extractMfccFeature(ppFileName)
            all_input_data.append(feature)
            all_input_class.append(Y[filename.split('.')[0]])

Walk on dir: genres Total File: 0
Walk on dir: genres/pop Total File: 100
Walk on dir: genres/blues Total File: 100
Walk on dir: genres/classical Total File: 100
Walk on dir: genres/country Total File: 100
Walk on dir: genres/hiphop Total File: 100
Walk on dir: genres/rock Total File: 100
Walk on dir: genres/reggae Total File: 100
Walk on dir: genres/metal Total File: 100
Walk on dir: genres/disco Total File: 100
Walk on dir: genres/jazz Total File: 100
CPU times: user 7min 57s, sys: 4min 18s, total: 12min 16s
Wall time: 6min 34s


### Shuffling all data

In [95]:
%%time
print("Shuffle input data to get better results")

all_input_shuffledData = np.asarray(all_input_shuffledData)
all_input_shuffledClass = np.asarray(all_input_shuffledClass)

s = np.arange(all_input_shuffledData.shape[0])
np.random.shuffle(s)

all_input_shuffledData = all_input_shuffledData[s]
all_input_shuffledClass = all_input_shuffledClass[s]

print("->all_input_shuffledData:",all_input_shuffledData.shape)
print("->all_input_shuffledClass:",all_input_shuffledClass.shape)

Shuffle input data to get better results
->all_input_shuffledData: (1000, 156)
->all_input_shuffledClass: (1000,)
CPU times: user 891 µs, sys: 375 µs, total: 1.27 ms
Wall time: 714 µs


### KNN

In [101]:
%%time
kf = KFold(n_splits=5)

for trainIndex, testIndex in kf.split(all_input_shuffledData):
    X_train, X_test = all_input_shuffledData[trainIndex], all_input_shuffledData[testIndex]
    y_train, y_test = all_input_shuffledClass[trainIndex], all_input_shuffledClass[testIndex]
    
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(X_train,y_train)
    k_score = knn.score(X_test,y_test)
    print("KNN Score: ", k_score)

KNN Score:  0.36
KNN Score:  0.37
KNN Score:  0.38
KNN Score:  0.37
KNN Score:  0.37
CPU times: user 172 ms, sys: 527 µs, total: 173 ms
Wall time: 170 ms


### SVM

In [102]:
%%time
kf = KFold(n_splits=5)

for trainIndex, testIndex in kf.split(all_input_data):
    X_train, X_test = all_input_shuffledData[trainIndex], all_input_shuffledData[testIndex]
    y_train, y_test = all_input_shuffledClass[trainIndex], all_input_shuffledClass[testIndex]
    
    svm1 = svm.LinearSVC(C=10, loss='squared_hinge', penalty='l2', tol=0.00001)
    svm1.fit(X_train,y_train)
    scoreSVM = svm1.score(X_test, y_test)
    print("Support Vector Machines: ", scoreSVM)   
   

Support Vector Machines:  0.455
Support Vector Machines:  0.355
Support Vector Machines:  0.45
Support Vector Machines:  0.48
Support Vector Machines:  0.47
CPU times: user 9.42 s, sys: 32.3 ms, total: 9.45 s
Wall time: 9.46 s


### Decision Tree

In [86]:
%%time
kf = KFold(n_splits=5)

for trainIndex, testIndex in kf.split(all_input_data):
    X_train, X_test = all_input_shuffledData[trainIndex], all_input_shuffledData[testIndex]
    y_train, y_test = all_input_shuffledClass[trainIndex], all_input_shuffledClass[testIndex]
    
    dt = DecisionTreeClassifier()
    dt.fit(X_train, y_train)
    score = regr_1.score(X_test, y_test)
    print("Decision tree score1: ", score)


Decision tree score1:  0.630862358543845
Decision tree score1:  0.6451124689271654
Decision tree score1:  0.5180215995884729
Decision tree score1:  0.6437727589261453
Decision tree score1:  0.6597542738543472
CPU times: user 716 ms, sys: 6.03 ms, total: 722 ms
Wall time: 716 ms


### XGBoost

In [87]:
kf = KFold(n_splits=5)

for trainIndex, testIndex in kf.split(all_input_data):
    X_train, X_test = all_input_shuffledData[trainIndex], all_input_shuffledData[testIndex]
    y_train, y_test = all_input_shuffledClass[trainIndex], all_input_shuffledClass[testIndex]
    
    xgb_params = {
        'eta': 0.05,
        'max_depth': 5,
        'subsample': 0.9,
        'objective': 'multi:softmax',
        'silent': 1,
        'colsample_bytree': 0.9,
        'num_class':10
    }

    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_test, y_test)
    
    model = xgb.train(xgb_params, d_train, num_boost_round = 240)
    pred_prob = model.predict(d_valid)
    print('Accuracy Score:',accuracy_score(y_test, pred_prob))


Accuracy Score: 0.595
Accuracy Score: 0.64
Accuracy Score: 0.66
Accuracy Score: 0.65
Accuracy Score: 0.54


In [12]:
# run garbage collector manually
gc.collect()

0